In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
import joblib
from skimage.io import imread
from skimage.transform import resize
 
def resize_all(src, pklname, width=360, height=180):
    """
    load images from path, resize them and write them as arrays to a dictionary, 
    together with labels and metadata. The dictionary is written to a pickle file 
    named '{pklname}_{width}x{height}px.pkl'.
     
    Parameter
    ---------
    src: str
        path to data
    pklname: str
        path to output file
    width: int
        target width of the image in pixels
    include: set[str]
        set containing str
    """
     
    height = height if height is not None else width
     
    data = dict()
    data['description'] = 'resized ({0}x{1})animal images in rgb'.format(int(width), int(height))
    data['label'] = []
    data['filename'] = []
    data['data'] = []   
     
    pklname = f"{pklname}_{width}x{height}px.pkl"
 
    # read all images in PATH, resize and write to DESTINATION_PATH
    for subdir in os.listdir(src):
        # if subdir in include:
        #     print(subdir)
        current_path = os.path.join(src, subdir)
 
        for file in os.listdir(current_path):
            if file[-4:] in {'jpeg'}:
                im = imread(os.path.join(current_path, file))
                im = resize(im, (width, height)) #[:,:,::-1]
                data['label'].append(subdir)
                data['filename'].append(file)
                data['data'].append(im)
 
        joblib.dump(data, pklname)

In [4]:
resize_all('/home/ducanh/project_master/final_project_cv/converted_dataset','/home/ducanh/project_master/final_project_cv/project_cv/sklearn_result' )

In [4]:
data_path = fr'{os.getenv("HOME")}/project_master/final_project_cv/converted_dataset'
os.listdir(data_path)

['0', '3', '1', '2', '4', '5']

In [5]:
data = joblib.load('sklearn_result_360x180px.pkl')
X = np.array(data['data'])
y = np.array(data['label'])

In [6]:
print(X.shape)

(2373, 360, 180, 3)


In [7]:
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import rescale
descriptors = []
for x in X:
    fed, dog_hog_img = hog(
        x, pixels_per_cell=(8,8), 
        cells_per_block=(2, 2), 
        orientations=9, 
        visualize=True, 
        block_norm='L2', 
        multichannel=True)
    descriptors.append(fed)

descriptors = np.array(descriptors)


/tmp/ipykernel_62901/119216922.py:6: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fed, dog_hog_img = hog(


In [ ]:
# print(len(descriptors))
# print(len(y))
print(descriptors.shape)

(2373, 10404)


In [ ]:

from sklearn import svm
from sklearn.metrics import accuracy_score
import ConfusionMatrixDisplay



def train(index, X_train, X_test, y_train, y_test):
    print("-------------------------------------")
    print(f"Running Train {index}")
    #Create a svm Classifier
    clf = svm.SVC(kernel='rbf',decision_function_shape = 'ovr', break_ties=True, probability=True) # Linear Kernel

    #Train the model using the training sets
    clf.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {acc}")
    clf.predict_proba(X_test)
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
    plt.show()
    plt.savefig(f'conf_mat_{index}.png')
    print("Saving confusion matrix")
    
    joblib.dump(clf, f'model_name_{index}.npy')


In [ ]:
from sklearn.model_selection import train_test_split

X, X_test, y, y_test = train_test_split(
     X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for i, train_index, val_index in enumerate(kf.split(X)):
    print("TRAIN:", train_index, "VAL:", val_index)
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    train(i, X_train, X_val, y_train, y_val)
    

In [13]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage import color
from imutils.object_detection import non_max_suppression
import numpy as np
import cv2
import os
import joblib

#Define HOG Parameters
# change them if necessary to orientations = 8, pixels per cell = (16,16), cells per block to (1,1) for weaker HOG
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3

# define the sliding window:
def sliding_window(image, stepSize, windowSize):# image is the input, step size is the no.of pixels needed to skip and windowSize is the size of the actual window
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):# this line and the line below actually defines the sliding part and loops over the x and y coordinates
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])
#%%
# Upload the saved svm model:
model = joblib.load('model_name.npy')

# Test the trained classifier on an image below!
scale = 0
detections = []
# read the image you want to detect the object in:
img= cv2.imread("9.jpeg")

# Try it with image resized if the image is too big
# img= cv2.resize(img,(300,200)) # can change the size to default by commenting this code out our put in a random number

# defining the size of the sliding window (has to be, same as the size of the image in the training data)
(winW, winH)= (360,180)
windowSize=(winW,winH)
downscale=2
# Apply sliding window:
for resized in pyramid_gaussian(img, downscale=2,multichannel = True): # loop over each layer of the image that you take!
    # loop over the sliding window for each layer of the pyramid
    for (x,y,window) in sliding_window(resized, stepSize=20, windowSize=(winW,winH)):
        # if the window does not meet our desired window size, ignore it!
        if window.shape[0] != winH or window.shape[1] !=winW: # ensure the sliding window has met the minimum size requirement
            continue
        # window=color.rgb2gray(window)
        fds, img_hog = hog(
                window, pixels_per_cell=(8,8), 
                cells_per_block=(2, 2), 
                orientations=9, 
                visualize=True, 
                block_norm='L2', 
                multichannel=True)
        fds = fds.reshape(1, -1) # re shape the image to make a silouhette of hog
        pred = model.predict(fds) # use the SVM model to make a prediction on the HOG features extracted from the window
        output = model.predict_proba(fds)
        output = np.array([output])
        # print(output.max())
        if output.max() > 0.6:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
            print("Detection:: Location -> ({}, {})".format(x, y))
            print("Scale ->  {} | Confidence Score {} \n".format(scale,output.max()))
            detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), output.max(),
                                int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
                                    int(windowSize[1]*(downscale**scale))))
    scale+=1
    
# clone = resized.copy()
# for (x_tl, y_tl, _, w, h) in detections:
#     cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
sc = [score for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
print(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.5)

# the peice of code above creates a raw bounding box prior to using NMS
# the code below creates a bounding box after using nms on the detections
# you can choose which one you want to visualise, as you deem fit... simply use the following function:
# cv2.imshow in this right place (since python is procedural it will go through the code line by line).
        
for (xA, yA, xB, yB) in pick:
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
cv2.imshow("Raw Detections after NMS", img)
#### Save the images below
k = cv2.waitKey(0) & 0xFF 
if k == 27:             #wait for ESC key to exit
    cv2.destroyAllWindows()
elif k == ord('s'):
    cv2.imwrite('saved_image_1.png',img)
    cv2.destroyAllWindows()

/tmp/ipykernel_62901/1983484597.py:42: FutureWarning: `multichannel` is a deprecated argument name for `pyramid_gaussian`. It will be removed in version 1.0. Please use `channel_axis` instead.
  for resized in pyramid_gaussian(img, downscale=2,multichannel = True): # loop over each layer of the image that you take!
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(


Detection:: Location -> (0, 0)
Scale ->  0 | Confidence Score 0.7687738704315673 

Detection:: Location -> (20, 0)
Scale ->  0 | Confidence Score 0.7535402675402052 



/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(


Detection:: Location -> (40, 0)
Scale ->  0 | Confidence Score 0.7720873104505651 

Detection:: Location -> (60, 0)
Scale ->  0 | Confidence Score 0.7498258667432512 



/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(


...

/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(


Detection:: Location -> (220, 300)
Scale ->  1 | Confidence Score 0.7277042090376797 

Detection:: Location -> (240, 300)
Scale ->  1 | Confidence Score 0.6546624693987932 



/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(
/tmp/ipykernel_62901/1983484597.py:49: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fds, img_hog = hog(


detection confidence score:  [0.7687738704315673, 0.7535402675402052, 0.7720873104505651, 0.7498258667432512, 0.6832436655704274, 0.6332811125071914, 0.602023123939631, 0.6036290090680199, 0.659227489515463, 0.7257542304617054, 0.7681697806292974, 0.7609449045768549, 0.7315410717527183, 0.6657349681772317, 0.6746584137914845, 0.667813583894959, 0.6688786760433196, 0.7040445780338149, 0.613822744448264, 0.6194384881434856, 0.6826767879166689, 0.6481399732741074, 0.6055416696980352, 0.6847944558509584, 0.6697097351284771, 0.7154006987946788, 0.6729024697676286, 0.6841517888848463, 0.7654656260461182, 0.7557571857349497, 0.7890984573697805, 0.8085539913922974, 0.8040173503631719, 0.7972728705389032, 0.7777334038310996, 0.7919381989712705, 0.835816390817206, 0.8323058414943227, 0.8114646020464783, 0.7724585693812245, 0.6707419681517054, 0.7516200944369495, 0.737711792338528, 0.7101529618293774, 0.6598893124770002, 0.6496990213925374, 0.6208773786099894, 0.658591573302385, 0.654925054326460

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to target thread (0x561f5e70e950)

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to target thread (0x561f5e70e950)

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to target thread (0x561f5e70e950)

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to target thread (0x561f5e70e950)

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to target thread (0x561f5e70e950)

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to target thread (0x561f5e70e950)

QObject::moveToThread: Current thread (0x561f5e70e950) is not the object's thread (0x561f3ca154d0).
Cannot move to tar

In [ ]:
from slidingwindow import sliding_window
from imutils.object_detection import non_max_suppression
import cv2
from skimage.transform import pyramid_gaussian


img = cv2.imread('9.jpeg')
(winW, winH)= (220,220)
windowSize=(winW,winH)
downscale=2
scale = 0
detections = []
for resize in pyramid_gaussian(image=img, downscale=2, multichannel = True):
    print(resize.shape)
    for (x,y,window) in sliding_window(resize, stepSize = 20, windowSize=(winW,winH)):
        if window.shape[0] != winH or window.shape[1] !=winW: # ensure the sliding window has met the minimum size requirement
            continue
        # image = tf.expand_dims(window, axis=0)
        # input_arr = tf.keras.preprocessing.image.img_to_array(image)
        # input_arr = np.array([input_arr])  # Convert single image to a batch.
        # predictions = load_model.predict(image)
        # print(predictions)
        # predictions = np.array(predictions)
        # output = softmax(predictions)
        # print(output)
        # window = color.rgb2gray(window)
        # print(window)
        # loss, = load_model(window)
        # if output.max() > 0.9:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
        #         print("Detection:: Location -> ({}, {})".format(x, y))
        #         print("Scale ->  {} | Confidence Score {} | Class {}\n".format(scale, output.max(), np.argmax(output)))
        #         detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), output.max(),
        #                            int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
        #                               int(windowSize[1]*(downscale**scale))))
       
    scale+=1


# for (x_tl, y_tl, _, w, h) in detections:
#     cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
sc = [score for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
pick = non_max_suppression(rects, probs = sc, overlapThresh = 0.3)

(995, 1280, 3)
(498, 640, 3)
(249, 320, 3)
(125, 160, 3)
(63, 80, 3)
(32, 40, 3)
(16, 20, 3)
(8, 10, 3)
(4, 5, 3)
(2, 3, 3)
(1, 2, 3)
(1, 1, 3)
detection confidence score:  []


/tmp/ipykernel_33190/439079968.py:13: FutureWarning: `multichannel` is a deprecated argument name for `pyramid_gaussian`. It will be removed in version 1.0. Please use `channel_axis` instead.
  for resize in pyramid_gaussian(image=img, downscale=2, multichannel = True):
